# Exporting data from BigQuery to Google Cloud Storage

In this notebook, we export BigQuery data to GCS so that we can reuse our Keras model that was developed on CSV data.

In [ ]:
# Run the chown command to change the ownership of the repository
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [ ]:
# Install the Google Cloud BigQuery library
%pip install google-cloud-bigquery==1.25.0

|████████████████████████████████| 169 kB 4.3 MB/s eta 0:00:01
Attempting uninstall: google-resumable-media
Found existing installation: google-resumable-media 0.7.1
Uninstalling google-resumable-media-0.7.1:
Successfully uninstalled google-resumable-media-0.7.1
Attempting uninstall: google-cloud-bigquery
Found existing installation: google-cloud-bigquery 1.26.1
Uninstalling google-cloud-bigquery-1.26.1:
Successfully uninstalled google-cloud-bigquery-1.26.1
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.
We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.
google-cloud-storage 1.30.0 requires google-resumable-media<2.0dev,>=0.6.0, but you'll have google-resumable-media 0.5.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


Please ignore any incompatibility warnings and errors.
**Restart** the kernel to use updated packages. (On the Notebook menu, select Kernel > Restart Kernel > Restart).


In [ ]:
# The OS module in python provides functions for interacting with the operating system.
import os

from google.cloud import bigquery

Change the following cell as necessary:

In [ ]:
# Change with your own bucket and project below:
BUCKET =  "<BUCKET>"
PROJECT = "<PROJECT>"

OUTDIR = "gs://{bucket}/taxifare/data".format(bucket=BUCKET)

os.environ['BUCKET'] = BUCKET
os.environ['OUTDIR'] = OUTDIR
os.environ['PROJECT'] = PROJECT

## Create BigQuery tables

If you haven not already created a BigQuery dataset for our data, run the following cell:

In [ ]:
# Initialize a BigQuery client.
bq = bigquery.Client(project = PROJECT)
dataset = bigquery.Dataset(bq.dataset("taxifare"))

# Create a new dataset with the `create_dataset()` method.
try:
    bq.create_dataset(dataset)
    print("Dataset created")
except:
    print("Dataset already exists")

Dataset created


Let's create a table with 1 million examples.

Note that the order of columns is exactly what was in our CSV files.

In [ ]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_training_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `nyc-tlc.yellow.trips`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 1000)) = 1
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

Make the validation dataset be 1/10 the size of the training dataset.

In [ ]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_valid_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `nyc-tlc.yellow.trips`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 10000)) = 2
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

## Export the tables as CSV files

In [ ]:
%%bash

echo "Deleting current contents of $OUTDIR"
gsutil -m -q rm -rf $OUTDIR

echo "Extracting training data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_training_data \
   $OUTDIR/taxi-train-*.csv

echo "Extracting validation data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_valid_data \
   $OUTDIR/taxi-valid-*.csv

# With `-l` option, gsutil will output additional information about each matching provider, bucket, subdirectory, or object.
gsutil ls -l $OUTDIR

Deleting current contents of gs://qwiklabs-gcp-04-e306f1f098e8/taxifare/data
Extracting training data to gs://qwiklabs-gcp-04-e306f1f098e8/taxifare/data
Extracting validation data to gs://qwiklabs-gcp-04-e306f1f098e8/taxifare/data
88345235  2020-09-15T09:55:47Z  gs://qwiklabs-gcp-04-e306f1f098e8/taxifare/data/taxi-train-000000000000.csv
8725746  2020-09-15T09:55:52Z  gs://qwiklabs-gcp-04-e306f1f098e8/taxifare/data/taxi-valid-000000000000.csv
TOTAL: 2 objects, 97070981 bytes (92.57 MiB)
CommandException: 1 files/objects could not be removed.
Waiting on bqjob_r11b700fae4476418_000001749130dde4_1 ... (14s) Current status: DONE
Waiting on bqjob_r3fd8761a598d7b29_00000174913121d1_1 ... (2s) Current status: DONE


In [ ]:
# The cat command outputs the contents of one or more URLs to stdout.
!gsutil cat gs://$BUCKET/taxifare/data/taxi-train-000000000000.csv | head -2

25.5,2013-01-11 10:33:06 UTC,-73.987038,40.693196,-73.90434,40.721355,2,unused
22.27,2009-12-24 06:56:00 UTC,-73.951928,40.781368,-73.870262,40.773127,2,unused


Copyright 2020 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.